In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import torch
from pytorchlb.unet import UNet    # Fix this!

unet = UNet(3, 2)
hooks_act = Hooks(unet, [unet.l1_, unet.l2_, unet.l3_, unet.l4_], calculate_stats(True))
hooks_wei = Hooks(unet, [unet.l1_.dconv[0], unet.l2_.dconv[0], unet.l3_.dconv[0], unet.l4_.dconv[0]], 
                  calculate_stats(False, True))

xb = torch.randn([2, 3, 128, 128])
pred = unet(xb)

In [ ]:
hooks_wei.storage